<a href="https://colab.research.google.com/github/AnumandlaS/Soccer-Player-Reidentification/blob/main/CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics opencv-python torch torchvision scipy numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial import distance
import torch
from torchvision.models import resnet18
from torchvision import transforms
from PIL import Image
import json

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Load the fine-tuned YOLOv11 model
model = YOLO("/content/best.pt")  # Replace with your .pt file path

# Load videos
broadcast_cap = cv2.VideoCapture("/content/broadcast.mp4")
tacticam_cap = cv2.VideoCapture("/content/tacticam.mp4")

In [4]:
# Check if videos opened successfully
if not broadcast_cap.isOpened() or not tacticam_cap.isOpened():
    print("Error: Could not open one or both video files.")
    exit()

In [5]:
# Initialize ResNet for appearance feature extraction
resnet = resnet18(pretrained=True).eval().cuda() if torch.cuda.is_available() else resnet18(pretrained=True).eval()
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 146MB/s]


In [6]:
# Store player tracks and features
broadcast_tracks = {}  # {track_id: [frames, bboxes, features]}
tacticam_tracks = {}   # {track_id: [frames, bboxes, features]}
player_mapping = {}    # {tacticam_id: broadcast_id}

In [7]:
# Function to extract appearance features from a bounding box
def extract_features(frame, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    crop = frame[y1:y2, x1:x2]
    if crop.size == 0:
        return None
    crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    crop = Image.fromarray(crop)
    crop = preprocess(crop).unsqueeze(0)
    if torch.cuda.is_available():
        crop = crop.cuda()
    with torch.no_grad():
        features = resnet(crop).cpu().numpy().flatten()
    return features

In [8]:
# Process videos frame by frame
frame_idx = 0
while broadcast_cap.isOpened() and tacticam_cap.isOpened():
    ret_b, frame_b = broadcast_cap.read()
    ret_t, frame_t = tacticam_cap.read()

    if not ret_b or not ret_t:
        break  # End of one or both videos

    # Detect and track players in broadcast video
    results_b = model.track(frame_b, persist=True, tracker="bytetrack.yaml")
    for det in results_b[0].boxes:
        if det.id is None:
            continue
        track_id = int(det.id)
        bbox = det.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2]
        conf = det.conf.cpu().numpy()
        if conf < 0.5:  # Confidence threshold
            continue
        features = extract_features(frame_b, bbox)
        if features is None:
            continue
        if track_id not in broadcast_tracks:
            broadcast_tracks[track_id] = {"frames": [], "bboxes": [], "features": []}
        broadcast_tracks[track_id]["frames"].append(frame_idx)
        broadcast_tracks[track_id]["bboxes"].append(bbox)
        broadcast_tracks[track_id]["features"].append(features)

    # Detect and track players in tacticam video
    results_t = model.track(frame_t, persist=True, tracker="bytetrack.yaml")
    for det in results_t[0].boxes:
        if det.id is None:
            continue
        track_id = int(det.id)
        bbox = det.xyxy[0].cpu().numpy()
        conf = det.conf.cpu().numpy()
        if conf < 0.5:
            continue
        features = extract_features(frame_t, bbox)
        if features is None:
            continue
        if track_id not in tacticam_tracks:
            tacticam_tracks[track_id] = {"frames": [], "bboxes": [], "features": []}
        tacticam_tracks[track_id]["frames"].append(frame_idx)
        tacticam_tracks[track_id]["bboxes"].append(bbox)
        tacticam_tracks[track_id]["features"].append(features)

    frame_idx += 1

# Release video captures
broadcast_cap.release()
tacticam_cap.release()

# Match players across videos based on appearance features
for t_id, t_data in tacticam_tracks.items():
    t_features = np.mean(t_data["features"], axis=0)  # Average features over frames
    min_dist = float("inf")
    best_match = None
    for b_id, b_data in broadcast_tracks.items():
        b_features = np.mean(b_data["features"], axis=0)
        dist = distance.cosine(t_features, b_features)
        if dist < min_dist:
            min_dist = dist
            best_match = b_id
    if min_dist < 0.5:  # Threshold for feature similarity
        player_mapping[t_id] = best_match

# Save mapping results
with open("player_mapping.json", "w") as f:
    json.dump(player_mapping, f, indent=4)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 3 players, 67.4ms
Speed: 16.5ms preprocess, 67.4ms inference, 392.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 3 referees, 55.5ms
Speed: 2.8ms preprocess, 55.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 55.6ms
Speed: 5.1ms preprocess, 55.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 2 referees, 55.5ms
Speed: 2.9ms preprocess, 55.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 players, 42.0ms
Speed: 3.8ms preprocess, 42.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 41.9ms
Speed: 3.0ms preprocess, 41.9ms inference, 1.4ms postprocess per image at shap

In [9]:
# Optional: Visualize results by annotating videos
def annotate_video(video_path, tracks, output_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        for track_id, data in tracks.items():
            for f_idx, bbox in zip(data["frames"], data["bboxes"]):
                if f_idx == frame_idx:
                    x1, y1, x2, y2 = map(int, bbox)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID: {track_id}", (x1, y1-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()


In [10]:
annotate_video("broadcast.mp4", broadcast_tracks, "broadcast_annotated.mp4")
annotate_video("tacticam.mp4", tacticam_tracks, "tacticam_annotated.mp4")

print("Player mapping completed. Results saved in 'player_mapping.json'.")
print("Annotated videos saved as 'broadcast_annotated.mp4' and 'tacticam_annotated.mp4'.")

Player mapping completed. Results saved in 'player_mapping.json'.
Annotated videos saved as 'broadcast_annotated.mp4' and 'tacticam_annotated.mp4'.
